In [7]:
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install beautifulsoup4


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.7 MB/s eta 0:00:00
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53725 sha256=54cffb8de7eb67baee593834da88a582e3487ca8612f93cbd1a6001bda91015b
  Stored in directory: /root/.cache/pip/wheels/e1/26/51/d0bffb3d2fd82256676d7ad3003faea3bd6dddc9577af665f4
Successfully built pypika
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependen

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [9]:
import requests
from bs4 import BeautifulSoup
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.chroma import Chroma
from sentence_transformers import SentenceTransformer

In [10]:
##Retrieving the HTML page from the URL of the NIKE fiscal report.

url = "https://www.businesswire.com/news/home/20240627955686/en/"
response = requests.get(url)


if response.status_code == 200:
    html_content = response.text
else:
    raise Exception(f"Failed to fetch the HTML content. Status code: {response.status_code}")


##Parsing the HTML and extracting text
soup = BeautifulSoup(html_content, 'html.parser')
paragraphs = soup.find_all('p')

extracted_text = "\n".join([para.get_text() for para in paragraphs])


##Checking the extracted content
print("Extracted Text:", extracted_text[:500])

Extracted Text: BEAVERTON, Ore.--(BUSINESS WIRE)--NIKE, Inc. (NYSE:NKE) today reported financial results for its fiscal 2024 fourth quarter and full year ended May 31, 2024.




"We are taking our near-term challenges head-on, while making continued progress in the areas that matter most to NIKE's future – serving the athlete through performance innovation, moving at the pace of the consumer and growing the complete marketplace," said John Donahoe, President & CEO, NIKE, Inc. "I'm confident that our teams are l


In [11]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=500,  # Setting the length of each chunk
    chunk_overlap=20  # Overlapping between consecutive chunks inorder to get the context
)

text_chunks = text_splitter.split_text(extracted_text) ##Splitting the Extracted Text into Chunks

print(f"Number of Chunks Created: {len(text_chunks)}")
print("Sample Chunks:")
for i, chunk in enumerate(text_chunks[:2]):  # Displaying first 2 chunks as a sample
    print(f"\nChunk {i + 1}:\n{chunk}")


Number of Chunks Created: 41
Sample Chunks:

Chunk 1:
BEAVERTON, Ore.--(BUSINESS WIRE)--NIKE, Inc. (NYSE:NKE) today reported financial results for its fiscal 2024 fourth quarter and full year ended May 31, 2024.

Chunk 2:
"We are taking our near-term challenges head-on, while making continued progress in the areas that matter most to NIKE's future – serving the athlete through performance innovation, moving at the pace of the consumer and growing the complete marketplace," said John Donahoe, President & CEO, NIKE, Inc. "I'm confident that our teams are lining up our competitive advantages to create greater impact for our business."**


In [12]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") ##using pre-trained huggingFace model

##Creating database for chunks with directory where embeddings and data will be saved using pre-trained model used to generate embeddings
chroma_db = Chroma(persist_directory="chroma_db", embedding_function=embedding_model)

# Generating embeddings and storing in Chroma
chroma_db.add_texts(texts=text_chunks)  ##adding text chunks to Chroma
chroma_db.persist() ##saving

print("Text chunks embedded and stored successfully!")


<ipython-input-12-b909d7bf6c87>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") ##using pre-trained huggingFace model
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to acce

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-12-b909d7bf6c87>:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(persist_directory="chroma_db", embedding_function=embedding_model)


Text chunks embedded and stored successfully!


<ipython-input-12-b909d7bf6c87>:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_db.persist() ##saving


In [13]:
# Perform a semantic search query on the Chroma vector database
query = "What were the key highlights of NIKE’s fourth quarter performance?"  # Example query
results = chroma_db.similarity_search(query=query, k=1)  # Retrieve top 2 relevant chunks

# Display the retrieved results
print("Query Results:")
for i, result in enumerate(results):
    print(f"\nResult {i + 1}:\n{result}")


Query Results:

Result 1:
page_content='Matthew Friend, Executive Vice President & Chief Financial Officer, NIKE, Inc. said, "We are driving better balance across our portfolio. While we are encouraged by our progress, our fourth quarter results highlighted challenges that have led us to update our Fiscal '25 outlook. We are taking actions to reposition NIKE to be more competitive, and to drive sustainable, profitable long-term growth."**

Fourth Quarter Income Statement Review


Fiscal 2024 Income Statement Review'


In [14]:
# Perform a semantic search query on the Chroma vector database
query = "How did NIKE’s full-year revenue compare across different markets?"  # Example query
results = chroma_db.similarity_search(query=query, k=1)  # Retrieve top 2 relevant chunks

# Display the retrieved results
print("Query Results:")
for i, result in enumerate(results):
    print(f"\nResult {i + 1}:\n{result}")


Query Results:

Result 1:
page_content='12%

Total

 

1,705

 

 

1,696

 

1%

4%

 

6,729

 

 

6,431

 

5%

5%

Global Brand Divisions2


 

11

 

 

14

 

-21%

-28%

 

45

 

 

58

 

-22%

-25%

TOTAL NIKE BRAND


 

12,149


 

 

12,225


 

-1%


1%


 

49,322


 

 

48,763


 

1%


1%


Converse

 

480

 

 

586

 

-18%

-17%

 

2,082

 

 

2,427

 

-14%

-15%

Corporate3


 

(23

)

 

14

 

—

—

 

(42

)

 

27

 

—

—

TOTAL NIKE, INC. REVENUES


$


12,606


 

$


12,825


 

-2%


0%


$


51,362'


In [15]:
# Perform a semantic search query on the Chroma vector database
query = "What is NIKE’s outlook for the next fiscal year?"  # Example query
results = chroma_db.similarity_search(query=query, k=1)  # Retrieve top 2 relevant chunks

# Display the retrieved results
print("Query Results:")
for i, result in enumerate(results):
    print(f"\nResult {i + 1}:\n{result}")


Query Results:

Result 1:
page_content='Matthew Friend, Executive Vice President & Chief Financial Officer, NIKE, Inc. said, "We are driving better balance across our portfolio. While we are encouraged by our progress, our fourth quarter results highlighted challenges that have led us to update our Fiscal '25 outlook. We are taking actions to reposition NIKE to be more competitive, and to drive sustainable, profitable long-term growth."**

Fourth Quarter Income Statement Review


Fiscal 2024 Income Statement Review'
